In [ ]:
import numpy as np
import aniposelib
import toml
import pandas as pd
from aniposelib.boards import CharucoBoard, Checkerboard
from aniposelib.cameras import Camera, CameraGroup
from aniposelib.utils import load_pose2d_fnames

import os

In [ ]:
# calibrate the cameras for 3d

do_cali = 0

#vidnames = [['camera-1_new_position_20221107.mp4'],
#            ['camera-2_new_position_20221107.mp4'],
#            ['camera-3_new_position_20221107.mp4']]
vidnames = [['camera-1_new_position_20221109_merge.mp4'],
            ['camera-2_new_position_20221109_merge.mp4'],
            ['camera-3_new_position_20221109_merge.mp4']]

cam_names = ['1', '2', '3']

n_cams = len(vidnames)

board = Checkerboard(8, 6,
                     square_length=1, # here, in mm but any unit works
                     manually_verify=False)


# the videos provided are fisheye, so we need the fisheye option
cgroup = CameraGroup.from_names(cam_names, fisheye=False)



In [ ]:
if do_cali: 
    # this will take about 15 minutes (mostly due to detection)
    # it will detect the charuco board in the videos,
    # then calibrate the cameras based on the detections, using iterative bundle adjustment
    cgroup.calibrate_videos(vidnames, board)

    # if you need to save and load
    # example saving and loading for later
    cgroup.dump('calibration.toml')

In [ ]:
if not do_cali:
    ## example of loading calibration from a file
    ## you can also load the provided file if you don't want to wait 15 minutes
    cgroup = CameraGroup.load('calibration.toml')


In [ ]:
lever_locs_all = {'camera-1':{('dodson'):np.array([645, 600]),('scorch'):np.array([425, 435])},
                  'camera-2':{('dodson'):np.array([1335,715]),('scorch'):np.array([550, 715])},
                  'camera-3':{('dodson'):np.array([1580,440]),('scorch'):np.array([1296,540])}}
tube_locs_all = {'camera-1':{('dodson'):np.array([1350,630]),('scorch'):np.array([555, 345])},
                 'camera-2':{('dodson'):np.array([1550,515]),('scorch'):np.array([350, 515])},
                 'camera-3':{('dodson'):np.array([1470,375]),('scorch'):np.array([805,475])}}
boxCorner1_locs_all = {'camera-1':{('dodson'):np.array([673, 421]),('scorch'):np.array([659, 408])},
                       'camera-2':{('dodson'):np.array([936, 527]),('scorch'):np.array([902, 526])},
                       'camera-3':{('dodson'):np.array([1323,395]),('scorch'):np.array([1310,398])}}
boxCorner2_locs_all = {'camera-1':{('dodson'):np.array([700, 682]),('scorch'):np.array([679, 665])},
                       'camera-2':{('dodson'):np.array([951, 773]),('scorch'):np.array([912, 775])},
                       'camera-3':{('dodson'):np.array([1285,634]),('scorch'):np.array([1270,641])}}
boxCorner3_locs_all = {'camera-1':{('dodson'):np.array([1232,1070]),('scorch'):np.array([501,513])},
                       'camera-2':{('dodson'):np.array([1599, 748]),('scorch'):np.array([236,791])},
                       'camera-3':{('dodson'):np.array([1525, 528]),('scorch'):np.array([734,864])}}
boxCorner4_locs_all = {'camera-1':{('dodson'):np.array([1585, 698]),('scorch'):np.array([842,446])},
                       'camera-2':{('dodson'):np.array([1343, 424]),('scorch'):np.array([502,461])},
                       'camera-3':{('dodson'):np.array([1190, 426]),('scorch'):np.array([511,581])}}

In [ ]:
## example triangulation without filtering, should take < 15 seconds
date_tgt = '20221128'
animal1 = 'Eddie'
animal2 = 'Sparkle'
#date_tgt = '20230209'
#animal1 = 'Ginger'
#animal2 = 'Kanga'
singlecam_ana_type = "DLC_dlcrnetms5_marmoset_tracking_with_middle_cameraSep1shuffle1_150000"

current_dir = os.getcwd()
add_date_dir = current_dir+'/anipose_3d_h5_files/'+date_tgt+'_'+animal1+'_'+animal2
bodyparts_3d_anipose_file = add_date_dir+'/'+date_tgt+'_'+animal1+'_'+animal2+'_anipose.h5'
if os.path.exists(bodyparts_3d_anipose_file):
    do_3dconstruct = 0
else:
    do_3dconstruct = 1
# do_3dconstruct=1
    
if do_3dconstruct:   

    twocamera_videos_cam12 = "/gpfs/gibbs/pi/jadi/VideoTracker_SocialInter/test_video_cooperative_task_3d/"+date_tgt+"_"+animal1+"_"+animal2+"_camera12/"
    twocamera_videos_cam23 = "/gpfs/gibbs/pi/jadi/VideoTracker_SocialInter/test_video_cooperative_task_3d/"+date_tgt+"_"+animal1+"_"+animal2+"_camera23/"

    bodyparts_cam1_cam12 = twocamera_videos_cam12+date_tgt+"_"+animal1+"_"+animal2+"_camera-1"+singlecam_ana_type+"_el_filtered.h5"
    bodyparts_cam2_cam12 = twocamera_videos_cam12+date_tgt+"_"+animal1+"_"+animal2+"_camera-2"+singlecam_ana_type+"_el_filtered.h5"
    bodyparts_cam3_cam23 = twocamera_videos_cam23+date_tgt+"_"+animal1+"_"+animal2+"_camera-3"+singlecam_ana_type+"_el_filtered.h5"

    bodyparts_3d_cam12_DLC = twocamera_videos_cam12+date_tgt+"_"+animal1+"_"+animal2+"_weikang.h5"
    bodyparts_3d_cam23_DLC = twocamera_videos_cam23+date_tgt+"_"+animal1+"_"+animal2+"_weikang.h5"


    ## save the the h5 file separately for each animals and save them in the same folder for future purpose

    # animal 1 - "dodson"

    # dodson h5 files
    bodyparts_cam1_cam12_dodson = twocamera_videos_cam12+date_tgt+"_"+animal1+"_"+animal2+"_camera-1"+singlecam_ana_type+"_el_filtered_dodson.h5"
    bodyparts_cam2_cam12_dodson = twocamera_videos_cam12+date_tgt+"_"+animal1+"_"+animal2+"_camera-2"+singlecam_ana_type+"_el_filtered_dodson.h5"
    bodyparts_cam3_cam23_dodson = twocamera_videos_cam23+date_tgt+"_"+animal1+"_"+animal2+"_camera-3"+singlecam_ana_type+"_el_filtered_dodson.h5"

    # cam1 dodson
    bodyparts_cam1_cam12_data = pd.read_hdf(bodyparts_cam1_cam12)
    bodyparts_cam1_cam12_dodson_data = {}
    bodyparts_cam1_cam12_dodson_data[singlecam_ana_type]=bodyparts_cam1_cam12_data.loc[:,(singlecam_ana_type,'dodson')]
    bodyparts_cam1_cam12_dodson_data=pd.concat(bodyparts_cam1_cam12_dodson_data, axis=1)
    # add lever
    lever_x = np.ones((1,np.shape(bodyparts_cam1_cam12_dodson_data)[0]))*lever_locs_all['camera-1']['dodson'][0]
    lever_y = np.ones((1,np.shape(bodyparts_cam1_cam12_dodson_data)[0]))*lever_locs_all['camera-1']['dodson'][1]
    lever_likelihood = np.ones((1,np.shape(bodyparts_cam1_cam12_dodson_data)[0]))
    bodyparts_cam1_cam12_dodson_data[(singlecam_ana_type,'lever','x')]=lever_x[0]
    bodyparts_cam1_cam12_dodson_data[(singlecam_ana_type,'lever','y')]=lever_y[0]
    bodyparts_cam1_cam12_dodson_data[(singlecam_ana_type,'lever','likelihood')]=lever_likelihood[0]
    # add tube
    tube_x = np.ones((1,np.shape(bodyparts_cam1_cam12_dodson_data)[0]))*tube_locs_all['camera-1']['dodson'][0]
    tube_y = np.ones((1,np.shape(bodyparts_cam1_cam12_dodson_data)[0]))*tube_locs_all['camera-1']['dodson'][1]
    tube_likelihood = np.ones((1,np.shape(bodyparts_cam1_cam12_dodson_data)[0]))
    bodyparts_cam1_cam12_dodson_data[(singlecam_ana_type,'tube','x')]=tube_x[0]
    bodyparts_cam1_cam12_dodson_data[(singlecam_ana_type,'tube','y')]=tube_y[0]
    bodyparts_cam1_cam12_dodson_data[(singlecam_ana_type,'tube','likelihood')]=tube_likelihood[0]
    #
    bodyparts_cam1_cam12_dodson_data.to_hdf(bodyparts_cam1_cam12_dodson,key='tracks')

    # cam2 dodson
    bodyparts_cam2_cam12_data = pd.read_hdf(bodyparts_cam2_cam12)
    bodyparts_cam2_cam12_dodson_data = {}
    bodyparts_cam2_cam12_dodson_data[singlecam_ana_type]=bodyparts_cam2_cam12_data.loc[:,(singlecam_ana_type,'dodson')]
    bodyparts_cam2_cam12_dodson_data=pd.concat(bodyparts_cam2_cam12_dodson_data, axis=1)
    # add lever
    lever_x = np.ones((1,np.shape(bodyparts_cam2_cam12_dodson_data)[0]))*lever_locs_all['camera-2']['dodson'][0]
    lever_y = np.ones((1,np.shape(bodyparts_cam2_cam12_dodson_data)[0]))*lever_locs_all['camera-2']['dodson'][1]
    lever_likelihood = np.ones((1,np.shape(bodyparts_cam2_cam12_dodson_data)[0]))
    bodyparts_cam2_cam12_dodson_data[(singlecam_ana_type,'lever','x')]=lever_x[0]
    bodyparts_cam2_cam12_dodson_data[(singlecam_ana_type,'lever','y')]=lever_y[0]
    bodyparts_cam2_cam12_dodson_data[(singlecam_ana_type,'lever','likelihood')]=lever_likelihood[0]
    # add tube
    tube_x = np.ones((1,np.shape(bodyparts_cam2_cam12_dodson_data)[0]))*tube_locs_all['camera-2']['dodson'][0]
    tube_y = np.ones((1,np.shape(bodyparts_cam2_cam12_dodson_data)[0]))*tube_locs_all['camera-2']['dodson'][1]
    tube_likelihood = np.ones((1,np.shape(bodyparts_cam2_cam12_dodson_data)[0]))
    bodyparts_cam2_cam12_dodson_data[(singlecam_ana_type,'tube','x')]=tube_x[0]
    bodyparts_cam2_cam12_dodson_data[(singlecam_ana_type,'tube','y')]=tube_y[0]
    bodyparts_cam2_cam12_dodson_data[(singlecam_ana_type,'tube','likelihood')]=tube_likelihood[0]
    #
    bodyparts_cam2_cam12_dodson_data.to_hdf(bodyparts_cam2_cam12_dodson,key='tracks')

    # cam3 dodson
    bodyparts_cam3_cam23_data = pd.read_hdf(bodyparts_cam3_cam23)
    bodyparts_cam3_cam23_dodson_data = {}
    bodyparts_cam3_cam23_dodson_data[singlecam_ana_type]=bodyparts_cam3_cam23_data.loc[:,(singlecam_ana_type,'dodson')]
    bodyparts_cam3_cam23_dodson_data=pd.concat(bodyparts_cam3_cam23_dodson_data, axis=1)
    # add lever
    lever_x = np.ones((1,np.shape(bodyparts_cam3_cam23_dodson_data)[0]))*lever_locs_all['camera-3']['dodson'][0]
    lever_y = np.ones((1,np.shape(bodyparts_cam3_cam23_dodson_data)[0]))*lever_locs_all['camera-3']['dodson'][1]
    lever_likelihood = np.ones((1,np.shape(bodyparts_cam3_cam23_dodson_data)[0]))
    bodyparts_cam3_cam23_dodson_data[(singlecam_ana_type,'lever','x')]=lever_x[0]
    bodyparts_cam3_cam23_dodson_data[(singlecam_ana_type,'lever','y')]=lever_y[0]
    bodyparts_cam3_cam23_dodson_data[(singlecam_ana_type,'lever','likelihood')]=lever_likelihood[0]
    # add tube
    tube_x = np.ones((1,np.shape(bodyparts_cam3_cam23_dodson_data)[0]))*tube_locs_all['camera-3']['dodson'][0]
    tube_y = np.ones((1,np.shape(bodyparts_cam3_cam23_dodson_data)[0]))*tube_locs_all['camera-3']['dodson'][1]
    tube_likelihood = np.ones((1,np.shape(bodyparts_cam3_cam23_dodson_data)[0]))
    bodyparts_cam3_cam23_dodson_data[(singlecam_ana_type,'tube','x')]=tube_x[0]
    bodyparts_cam3_cam23_dodson_data[(singlecam_ana_type,'tube','y')]=tube_y[0]
    bodyparts_cam3_cam23_dodson_data[(singlecam_ana_type,'tube','likelihood')]=tube_likelihood[0]
    #
    bodyparts_cam3_cam23_dodson_data.to_hdf(bodyparts_cam3_cam23_dodson,key='tracks')


    # animal 2 - "scorch"

    # scorch h5 files
    bodyparts_cam1_cam12_scorch = twocamera_videos_cam12+date_tgt+"_"+animal1+"_"+animal2+"_camera-1"+singlecam_ana_type+"_el_filtered_scorch.h5"
    bodyparts_cam2_cam12_scorch = twocamera_videos_cam12+date_tgt+"_"+animal1+"_"+animal2+"_camera-2"+singlecam_ana_type+"_el_filtered_scorch.h5"
    bodyparts_cam3_cam23_scorch = twocamera_videos_cam23+date_tgt+"_"+animal1+"_"+animal2+"_camera-3"+singlecam_ana_type+"_el_filtered_scorch.h5"

    # cam1 scorch
    bodyparts_cam1_cam12_data = pd.read_hdf(bodyparts_cam1_cam12)
    bodyparts_cam1_cam12_scorch_data = {}
    bodyparts_cam1_cam12_scorch_data[singlecam_ana_type]=bodyparts_cam1_cam12_data.loc[:,(singlecam_ana_type,'scorch')]
    bodyparts_cam1_cam12_scorch_data=pd.concat(bodyparts_cam1_cam12_scorch_data, axis=1)
    # add lever
    lever_x = np.ones((1,np.shape(bodyparts_cam1_cam12_scorch_data)[0]))*lever_locs_all['camera-1']['scorch'][0]
    lever_y = np.ones((1,np.shape(bodyparts_cam1_cam12_scorch_data)[0]))*lever_locs_all['camera-1']['scorch'][1]
    lever_likelihood = np.ones((1,np.shape(bodyparts_cam1_cam12_scorch_data)[0]))
    bodyparts_cam1_cam12_scorch_data[(singlecam_ana_type,'lever','x')]=lever_x[0]
    bodyparts_cam1_cam12_scorch_data[(singlecam_ana_type,'lever','y')]=lever_y[0]
    bodyparts_cam1_cam12_scorch_data[(singlecam_ana_type,'lever','likelihood')]=lever_likelihood[0]
    # add tube
    tube_x = np.ones((1,np.shape(bodyparts_cam1_cam12_scorch_data)[0]))*tube_locs_all['camera-1']['scorch'][0]
    tube_y = np.ones((1,np.shape(bodyparts_cam1_cam12_scorch_data)[0]))*tube_locs_all['camera-1']['scorch'][1]
    tube_likelihood = np.ones((1,np.shape(bodyparts_cam1_cam12_scorch_data)[0]))
    bodyparts_cam1_cam12_scorch_data[(singlecam_ana_type,'tube','x')]=tube_x[0]
    bodyparts_cam1_cam12_scorch_data[(singlecam_ana_type,'tube','y')]=tube_y[0]
    bodyparts_cam1_cam12_scorch_data[(singlecam_ana_type,'tube','likelihood')]=tube_likelihood[0]
    #
    bodyparts_cam1_cam12_scorch_data.to_hdf(bodyparts_cam1_cam12_scorch,key='tracks')

    # cam2 scorch
    bodyparts_cam2_cam12_data = pd.read_hdf(bodyparts_cam2_cam12)
    bodyparts_cam2_cam12_scorch_data = {}
    bodyparts_cam2_cam12_scorch_data[singlecam_ana_type]=bodyparts_cam2_cam12_data.loc[:,(singlecam_ana_type,'scorch')]
    bodyparts_cam2_cam12_scorch_data=pd.concat(bodyparts_cam2_cam12_scorch_data, axis=1)
    # add lever
    lever_x = np.ones((1,np.shape(bodyparts_cam2_cam12_scorch_data)[0]))*lever_locs_all['camera-2']['scorch'][0]
    lever_y = np.ones((1,np.shape(bodyparts_cam2_cam12_scorch_data)[0]))*lever_locs_all['camera-2']['scorch'][1]
    lever_likelihood = np.ones((1,np.shape(bodyparts_cam2_cam12_scorch_data)[0]))
    bodyparts_cam2_cam12_scorch_data[(singlecam_ana_type,'lever','x')]=lever_x[0]
    bodyparts_cam2_cam12_scorch_data[(singlecam_ana_type,'lever','y')]=lever_y[0]
    bodyparts_cam2_cam12_scorch_data[(singlecam_ana_type,'lever','likelihood')]=lever_likelihood[0]
    # add tube
    tube_x = np.ones((1,np.shape(bodyparts_cam2_cam12_scorch_data)[0]))*tube_locs_all['camera-2']['scorch'][0]
    tube_y = np.ones((1,np.shape(bodyparts_cam2_cam12_scorch_data)[0]))*tube_locs_all['camera-2']['scorch'][1]
    tube_likelihood = np.ones((1,np.shape(bodyparts_cam2_cam12_scorch_data)[0]))
    bodyparts_cam2_cam12_scorch_data[(singlecam_ana_type,'tube','x')]=tube_x[0]
    bodyparts_cam2_cam12_scorch_data[(singlecam_ana_type,'tube','y')]=tube_y[0]
    bodyparts_cam2_cam12_scorch_data[(singlecam_ana_type,'tube','likelihood')]=tube_likelihood[0]
    #
    bodyparts_cam2_cam12_scorch_data.to_hdf(bodyparts_cam2_cam12_scorch,key='tracks')

    # cam3 scorch
    bodyparts_cam3_cam23_data = pd.read_hdf(bodyparts_cam3_cam23)
    bodyparts_cam3_cam23_scorch_data = {}
    bodyparts_cam3_cam23_scorch_data[singlecam_ana_type]=bodyparts_cam3_cam23_data.loc[:,(singlecam_ana_type,'scorch')]
    bodyparts_cam3_cam23_scorch_data=pd.concat(bodyparts_cam3_cam23_scorch_data, axis=1)
    # add lever
    lever_x = np.ones((1,np.shape(bodyparts_cam3_cam23_scorch_data)[0]))*lever_locs_all['camera-3']['scorch'][0]
    lever_y = np.ones((1,np.shape(bodyparts_cam3_cam23_scorch_data)[0]))*lever_locs_all['camera-3']['scorch'][1]
    lever_likelihood = np.ones((1,np.shape(bodyparts_cam3_cam23_scorch_data)[0]))
    bodyparts_cam3_cam23_scorch_data[(singlecam_ana_type,'lever','x')]=lever_x[0]
    bodyparts_cam3_cam23_scorch_data[(singlecam_ana_type,'lever','y')]=lever_y[0]
    bodyparts_cam3_cam23_scorch_data[(singlecam_ana_type,'lever','likelihood')]=lever_likelihood[0]
    # add tube
    tube_x = np.ones((1,np.shape(bodyparts_cam3_cam23_scorch_data)[0]))*tube_locs_all['camera-3']['scorch'][0]
    tube_y = np.ones((1,np.shape(bodyparts_cam3_cam23_scorch_data)[0]))*tube_locs_all['camera-3']['scorch'][1]
    tube_likelihood = np.ones((1,np.shape(bodyparts_cam3_cam23_scorch_data)[0]))
    bodyparts_cam3_cam23_scorch_data[(singlecam_ana_type,'tube','x')]=tube_x[0]
    bodyparts_cam3_cam23_scorch_data[(singlecam_ana_type,'tube','y')]=tube_y[0]
    bodyparts_cam3_cam23_scorch_data[(singlecam_ana_type,'tube','likelihood')]=tube_likelihood[0]
    #
    bodyparts_cam3_cam23_scorch_data.to_hdf(bodyparts_cam3_cam23_scorch,key='tracks')

In [ ]:
## read the 3d recontruction h5 file
if do_3dconstruct:
    bodyparts_3d_cam12_DLC_data = pd.read_hdf(bodyparts_3d_cam12_DLC)
    bodyparts_3d_cam12_DLC_data.head()

In [ ]:
## triangulation without filtering, should take < 15 seconds
if do_3dconstruct:
    # for the animal 1 - dodson
    fname_dict = {
        '1': bodyparts_cam1_cam12_dodson,
        '2': bodyparts_cam2_cam12_dodson,
        '3': bodyparts_cam3_cam23_dodson,
    }

    d = load_pose2d_fnames(fname_dict, cam_names=cgroup.get_names())

    score_threshold = 0.1

    n_cams, n_points, n_joints, _ = d['points'].shape
    points = d['points']
    scores = d['scores']

    bodyparts = d['bodyparts']

    # remove points that are below threshold
    points[scores < score_threshold] = np.nan

    points_flat = points.reshape(n_cams, -1, 2)
    scores_flat = scores.reshape(n_cams, -1)

    p3ds_flat = cgroup.triangulate(points_flat, progress=True)
    reprojerr_flat = cgroup.reprojection_error(p3ds_flat, points_flat, mean=True)

    p3ds = p3ds_flat.reshape(n_points, n_joints, 3)
    reprojerr = reprojerr_flat.reshape(n_points, n_joints)


In [ ]:
## save the new files - animal 1 dodson
if do_3dconstruct:
    bodyparts_3d_dodson_anipose = {}
    nbodyparts = np.shape(bodyparts)[0]
    for ibodypart in np.arange(0,nbodyparts,1):
        # remove outlier
        ind_outlier_x = (p3ds[:,ibodypart,0]<(np.nanmean(p3ds[:,ibodypart,0])-2*np.nanstd(p3ds[:,ibodypart,0])))|(p3ds[:,ibodypart,0]>(np.nanmean(p3ds[:,ibodypart,0])+2*np.nanstd(p3ds[:,ibodypart,0])))
        p3ds[ind_outlier_x,ibodypart,0]=np.nan
        ind_outlier_y = (p3ds[:,ibodypart,1]<(np.nanmean(p3ds[:,ibodypart,1])-2*np.nanstd(p3ds[:,ibodypart,1])))|(p3ds[:,ibodypart,1]>(np.nanmean(p3ds[:,ibodypart,1])+2*np.nanstd(p3ds[:,ibodypart,1])))
        p3ds[ind_outlier_y,ibodypart,1]=np.nan
        ind_outlier_z = (p3ds[:,ibodypart,2]<(np.nanmean(p3ds[:,ibodypart,2])-2*np.nanstd(p3ds[:,ibodypart,2])))|(p3ds[:,ibodypart,2]>(np.nanmean(p3ds[:,ibodypart,2])+2*np.nanstd(p3ds[:,ibodypart,2])))
        p3ds[ind_outlier_z,ibodypart,2]=np.nan

        bodyparts_3d_dodson_anipose[('weikang','dodson',bodyparts[ibodypart],'x')] = p3ds[:,ibodypart, 0]
        bodyparts_3d_dodson_anipose[('weikang','dodson',bodyparts[ibodypart],'y')] = p3ds[:,ibodypart, 1]
        bodyparts_3d_dodson_anipose[('weikang','dodson',bodyparts[ibodypart],'z')] = p3ds[:,ibodypart, 2]
    bodyparts_3d_dodson_anipose = pd.DataFrame(bodyparts_3d_dodson_anipose)

In [ ]:
## triangulation without filtering, should take < 15 seconds
if do_3dconstruct:
    # for the animal 1 - scorch
    fname_dict = {
        '1': bodyparts_cam1_cam12_scorch,
        '2': bodyparts_cam2_cam12_scorch,
        '3': bodyparts_cam3_cam23_scorch,
    }

    d = load_pose2d_fnames(fname_dict, cam_names=cgroup.get_names())

    score_threshold = 0.1

    n_cams, n_points, n_joints, _ = d['points'].shape
    points = d['points']
    scores = d['scores']

    bodyparts = d['bodyparts']

    # remove points that are below threshold
    points[scores < score_threshold] = np.nan

    points_flat = points.reshape(n_cams, -1, 2)
    scores_flat = scores.reshape(n_cams, -1)

    p3ds_flat = cgroup.triangulate(points_flat, progress=True)
    reprojerr_flat = cgroup.reprojection_error(p3ds_flat, points_flat, mean=True)

    p3ds = p3ds_flat.reshape(n_points, n_joints, 3)
    reprojerr = reprojerr_flat.reshape(n_points, n_joints)


In [ ]:
## save the new files - animal 2 scorch
if do_3dconstruct:
    bodyparts_3d_scorch_anipose = {}
    nbodyparts = np.shape(bodyparts)[0]
    for ibodypart in np.arange(0,nbodyparts,1):
        # remove outlier
        ind_outlier_x = (p3ds[:,ibodypart,0]<(np.nanmean(p3ds[:,ibodypart,0])-2*np.nanstd(p3ds[:,ibodypart,0])))|(p3ds[:,ibodypart,0]>(np.nanmean(p3ds[:,ibodypart,0])+2*np.nanstd(p3ds[:,ibodypart,0])))
        p3ds[ind_outlier_x,ibodypart,0]=np.nan
        ind_outlier_y = (p3ds[:,ibodypart,1]<(np.nanmean(p3ds[:,ibodypart,1])-2*np.nanstd(p3ds[:,ibodypart,1])))|(p3ds[:,ibodypart,1]>(np.nanmean(p3ds[:,ibodypart,1])+2*np.nanstd(p3ds[:,ibodypart,1])))
        p3ds[ind_outlier_y,ibodypart,1]=np.nan
        ind_outlier_z = (p3ds[:,ibodypart,2]<(np.nanmean(p3ds[:,ibodypart,2])-2*np.nanstd(p3ds[:,ibodypart,2])))|(p3ds[:,ibodypart,2]>(np.nanmean(p3ds[:,ibodypart,2])+2*np.nanstd(p3ds[:,ibodypart,2])))
        p3ds[ind_outlier_z,ibodypart,2]=np.nan
        
        bodyparts_3d_scorch_anipose[('weikang','scorch',bodyparts[ibodypart],'x')] = p3ds[:,ibodypart, 0]
        bodyparts_3d_scorch_anipose[('weikang','scorch',bodyparts[ibodypart],'y')] = p3ds[:,ibodypart, 1]
        bodyparts_3d_scorch_anipose[('weikang','scorch',bodyparts[ibodypart],'z')] = p3ds[:,ibodypart, 2]
    bodyparts_3d_scorch_anipose = pd.DataFrame(bodyparts_3d_scorch_anipose)

In [ ]:
## combine the two animals 
if do_3dconstruct:
    bodyparts_3d_anipose = pd.concat([bodyparts_3d_dodson_anipose,bodyparts_3d_scorch_anipose],axis=1)

    # save the combine the two animal 3d file
    import os
    current_dir = os.getcwd()
    add_date_dir = current_dir+'/anipose_3d_h5_files/'+date_tgt+'_'+animal1+'_'+animal2
    if not os.path.exists(add_date_dir):
        os.makedirs(add_date_dir)
    bodyparts_3d_anipose_file = add_date_dir+'/'+date_tgt+'_'+animal1+'_'+animal2+'_anipose.h5'
    bodyparts_3d_anipose.to_hdf(bodyparts_3d_anipose_file,key='tracks')

In [ ]:
## load saved 3d_anipose h5 file
if not do_3dconstruct:
    import os
    current_dir = os.getcwd()
    add_date_dir = current_dir+'/anipose_3d_h5_files/'+date_tgt+'_'+animal1+'_'+animal2
    bodyparts_3d_anipose_file = add_date_dir+'/'+date_tgt+'_'+animal1+'_'+animal2+'_anipose.h5'
    bodyparts_3d_anipose = pd.read_hdf(bodyparts_3d_anipose_file)

In [ ]:
bodyparts_3d_anipose

In [ ]:
# plot the x, y, z coordinates of joint 1

import matplotlib.pyplot as plt
%matplotlib notebook

plt.figure(figsize=(9.4, 6))
plt.plot(bodyparts_3d_anipose[('weikang','dodson','rightTuft','x')])
plt.plot(bodyparts_3d_anipose[('weikang','dodson','rightTuft','y')])
plt.plot(bodyparts_3d_anipose[('weikang','dodson','rightTuft','z')])
plt.xlabel("Time (frames)")
plt.ylabel("Coordinate (mm)")
plt.title("x, y, z coordinates of {}".format('rightTuft'))

In [ ]:
## plot the example videos

import sys
sys.path.append('/home/ws523/marmoset_tracking_DLCv2/following_up_analysis/3d_recontruction_analysis_self_and_coop_task/ana_functions')
from tracking_video_3d_demo import tracking_video_3d_demo

animalnames_videotrack = ['dodson','scorch']
bodypartnames_videotrack = ['rightTuft','whiteBlaze','leftTuft','rightEye','leftEye','mouth','lever','tube','boxCorner1','boxCorner2','boxCorner3','boxCorner4']
animal1_filename = animal1
animal2_filename = animal2
session_start_time = 1.00
fps = 30
nframes = 2*fps

withboxCorner = 1

current_dir = os.getcwd()
add_date_dir = current_dir+'/anipose_3d_demo_videos/'+date_tgt+'_'+animal1_filename+'_'+animal2_filename
if not os.path.exists(add_date_dir):
    os.makedirs(add_date_dir)
video_file = add_date_dir+'/'+date_tgt+'_'+animal1_filename+'_'+animal2_filename+'_anipose_3d_tracking_demo.mp4'

tracking_video_3d_demo(bodyparts_3d_anipose['weikang'],animalnames_videotrack,bodypartnames_videotrack,date_tgt,animal1_filename,animal2_filename,session_start_time,fps,nframes,video_file,withboxCorner)

In [ ]:
## plot the example videos
## for comparison make the video based on DLC's own 3d recontruction

twocamera_videos_cam12 = "/gpfs/gibbs/pi/jadi/VideoTracker_SocialInter/test_video_cooperative_task_3d/"+date_tgt+"_"+animal1+"_"+animal2+"_camera12/"
twocamera_videos_cam23 = "/gpfs/gibbs/pi/jadi/VideoTracker_SocialInter/test_video_cooperative_task_3d/"+date_tgt+"_"+animal1+"_"+animal2+"_camera23/"

bodyparts_cam1_cam12 = twocamera_videos_cam12+date_tgt+"_"+animal1+"_"+animal2+"_camera-1"+singlecam_ana_type+"_el_filtered.h5"
bodyparts_cam2_cam12 = twocamera_videos_cam12+date_tgt+"_"+animal1+"_"+animal2+"_camera-2"+singlecam_ana_type+"_el_filtered.h5"
bodyparts_cam3_cam23 = twocamera_videos_cam23+date_tgt+"_"+animal1+"_"+animal2+"_camera-3"+singlecam_ana_type+"_el_filtered.h5"

bodyparts_3d_cam12_DLC = twocamera_videos_cam12+date_tgt+"_"+animal1+"_"+animal2+"_weikang.h5"
bodyparts_3d_cam23_DLC = twocamera_videos_cam23+date_tgt+"_"+animal1+"_"+animal2+"_weikang.h5"

singlecam_ana_type = "DLC_dlcrnetms5_marmoset_tracking_with_middle_cameraSep1shuffle1_150000"

bodyparts_3d_cam12_DLC_data = pd.read_hdf(bodyparts_3d_cam12_DLC)
bodyparts_3d_cam23_DLC_data = pd.read_hdf(bodyparts_3d_cam23_DLC)


# camera 12 pair

import sys
sys.path.append('/home/ws523/marmoset_tracking_DLCv2/following_up_analysis/3d_recontruction_analysis_self_and_coop_task/ana_functions')
from tracking_video_3d_demo import tracking_video_3d_demo

animalnames_videotrack = ['dodson','scorch']
bodypartnames_videotrack = ['rightTuft','whiteBlaze','leftTuft','rightEye','leftEye','mouth']
animal1_filename = animal1
animal2_filename = animal2
session_start_time = 1.00
fps = 30
nframes = 60*fps

withboxCorner = 0

current_dir = os.getcwd()
add_date_dir = current_dir+'/DLC_cam12_3d_demo_videos/'+date_tgt+'_'+animal1_filename+'_'+animal2_filename
if not os.path.exists(add_date_dir):
    os.makedirs(add_date_dir)
video_file = add_date_dir+'/'+date_tgt+'_'+animal1_filename+'_'+animal2_filename+'_DLCcam12_3d_tracking_demo.mp4'

tracking_video_3d_demo(bodyparts_3d_cam12_DLC_data['weikang'],animalnames_videotrack,bodypartnames_videotrack,date_tgt,animal1_filename,animal2_filename,session_start_time,fps,nframes,video_file,withboxCorner)



# camera 23 pair

import sys
sys.path.append('/home/ws523/marmoset_tracking_DLCv2/following_up_analysis/3d_recontruction_analysis_self_and_coop_task/ana_functions')
from tracking_video_3d_demo import tracking_video_3d_demo

animalnames_videotrack = ['dodson','scorch']
bodypartnames_videotrack = ['rightTuft','whiteBlaze','leftTuft','rightEye','leftEye','mouth']
animal1_filename = animal1
animal2_filename = animal2
session_start_time = 1.00
fps = 30
nframes = 60*fps

withboxCorner = 0

current_dir = os.getcwd()
add_date_dir = current_dir+'/DLC_cam23_3d_demo_videos/'+date_tgt+'_'+animal1_filename+'_'+animal2_filename
if not os.path.exists(add_date_dir):
    os.makedirs(add_date_dir)
video_file = add_date_dir+'/'+date_tgt+'_'+animal1_filename+'_'+animal2_filename+'_DLCcam23_3d_tracking_demo.mp4'

tracking_video_3d_demo(bodyparts_3d_cam23_DLC_data['weikang'],animalnames_videotrack,bodypartnames_videotrack,date_tgt,animal1_filename,animal2_filename,session_start_time,fps,nframes,video_file,withboxCorner)







